In [1]:
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier

In [8]:
# Путь к папке с атаками
csvs_path = "csvs"

csvs_path_combined = "combined_csvs"
os.makedirs(csvs_path_combined, exist_ok=True)

traffic_types = os.listdir(csvs_path)
traffic_type_label = {attack: idx + 1 for idx, attack in enumerate(traffic_types)}
traffic_type_label

{'attack': 1, 'normal': 2}

In [9]:
# Найти все уникальные размеры выборок
packet_intervals = set()

for traffic_type in traffic_types:
    traffic_path = os.path.join(csvs_path, traffic_type)
    print(traffic_path)
    for filename in os.listdir(traffic_path):
        # Извлечение размера выборки из имени файла с помощью регулярного выражения
        match = re.search(r"_(\d+)_\d+_", filename)
        if match:
            packet_intervals.add(int(match.group(1)))

# Сортируем размеры выборок
packet_intervals = sorted(packet_intervals)
print("Найденные размеры выборок пакетов:", packet_intervals)

csvs\attack
csvs\normal
Найденные размеры выборок пакетов: [10, 50, 100, 250, 500, 750, 1000, 5000, 10000]


In [14]:
# Объединение данных для каждого размера выборки
for packet_size in packet_intervals:
    combined_df = pd.DataFrame()

    # Определяем единственный файл нормального и атакующего трафика для текущего размера выборки
    normal_file = next(
        (
            f
            for f in os.listdir(os.path.join(csvs_path, "normal"))
            if f"_{packet_size}_0_" in f
        ),
        None,
    )
    attack_file = next(
        (
            f
            for f in os.listdir(os.path.join(csvs_path, "attack"))
            if f"_{packet_size}_1_" in f
        ),
        None,
    )

    if normal_file and attack_file:
        # Чтение файла с нормальным трафиком и добавление метки
        normal_df = pd.read_csv(os.path.join(csvs_path, "normal", normal_file))
        normal_df["Label"] = 0  # Метка для нормального трафика
        combined_df = pd.concat([combined_df, normal_df], ignore_index=True)

        # Чтение файла с атакующим трафиком и добавление метки
        attack_df = pd.read_csv(os.path.join(csvs_path, "attack", attack_file))
        attack_df["Label"] = 1#traffic_type_label[traffic_type]  # Уникальная метка для атаки
        combined_df = pd.concat([combined_df, attack_df], ignore_index=True)

    # Сохранение объединенного CSV для текущего размера выборки
    combined_file = os.path.join(csvs_path_combined, f"combined_{packet_size}_packets.csv")
    combined_df.to_csv(combined_file, index=False)